# 2018-06-05 - Psychophysics using psychopy
On repasse sur un paradoxe plus classique de psychophysique.

On récupère les initiales :

In [1]:
exp_info = '2018-06-05_LP'
print (exp_info)

2018-06-05_LP


On défini la génération de MotionClouds :

In [2]:
import numpy as np
import MotionClouds as mc
import matplotlib.pyplot as plt
import os
import imageio
import random

downscale = 1
fig_width = 21
fx, fy, ft = mc.get_grids(mc.N_X/downscale, mc.N_Y/downscale, 1)

# generates a cloud of given theta and b_theta
def generate_random_cloud(theta, B_theta):
    mc_i = mc.envelope_gabor(fx, fy, ft, V_X=0., V_Y=0.,
                             B_V=0, theta=theta, B_theta=B_theta)
    im = mc.random_cloud(mc_i)
    im = (mc.rectif(im) * 255).astype('uint8')
    fname = '/tmp/%s_%s.png' % (theta, B_theta)
    imageio.imwrite(fname, im[:, :, 0])
    return fname

On définit les paramètres et on teste :

In [3]:
from psychopy import visual, core, event
import MotionClouds as MC

test_length = 150  # number of trials
#trial_length = 2.6  # length of trial, in second
fixation_length = .5  # length of fixation, in second
stim1_length = 0.25  # length of first stim, in second
stim2_length = 0.25  # length of the second stim, in second

fullscr = False

fullscr = True

# Psychopy window
win = visual.Window(fullscr=fullscr, color=[0,0,0], winType='pygame', monitor='testMonitor', units='height')
mouse = event.Mouse(visible=False)

fixation = visual.ShapeStim(vertices='cross', win=win, size=0.05, pos=[0, 0], fillColor='black', units='height')
fixation.autolog = False

MC1 = generate_random_cloud(np.pi/2, B_theta=np.pi/2)
bitmap1 = visual.ImageStim(win, MC1, mask='gauss', size=0.8, units='height')
bitmap1.autolog = False
bitmap2 = visual.ImageStim(win, MC1, mask='gauss', size=0.8, units='height')
bitmap2.autolog = False
#bitmap3 = visual.ImageStim(win, '%s.png' %
#                           str(2 * (np.pi/3)), mask='circle')
#bitmap3.autolog = False


#msg = visual.TextStim(win, text='<- LEFT or RIGHT ->', color='black')
msg = visual.TextStim(win, text='?', color='black')

#shift_dict = {'right': 2,
#              'left':  3}


# Answer list
ans_list = []
N_B_thetas = 5
B_thetas = np.pi*np.logspace(-7, 0, N_B_thetas, base=2)
    
std_theta = np.pi/6
for trial in range(test_length):
    clock = core.Clock()
    fixation.draw()
    win.flip()
    
    # Shift to left (3) or to right (2) ?
    #shift = random.randint(2, 3)
    theta = std_theta * np.random.randn()

    # MC generation
    B_theta = B_thetas[random.randint(0, N_B_thetas-1)]

    MC1 = generate_random_cloud(np.pi/2, B_theta=B_theta)
    bitmap1.setImage(MC1)

    MC2 = generate_random_cloud(np.pi/2 - theta, B_theta=B_theta)  # if shift = 2
    bitmap2.setImage(MC2)

    #MC3 = generate_random_cloud(2 * (np.pi/3), B_theta=B_theta)  # if shift = 3
    # MC to psychopy objects
    
    # Times the trial
    while clock.getTime() < fixation_length + stim1_length + stim2_length:
        if 0.0 <= clock.getTime() < fixation_length:  # fixation
            fixation.draw()
        if fixation_length + 0.01 <= clock.getTime() < fixation_length + stim1_length:
            bitmap1.draw()
        if fixation_length + stim1_length <= clock.getTime() < fixation_length + stim1_length + stim2_length:
            bitmap2.draw()
        win.flip()

    # display orientation choice
    msg.draw()
    win.flip()

    # answer using the keyboard
    while True:
        ans = event.waitKeys()

        if ans in ['escape', 'q']:
            win.close()
            core.quit()
        if len(ans) > 0:
            print(trial, theta*180/np.pi, ans[0], B_theta*180/np.pi)
            break

    # Output shape per trial is : trial number, shift direction, answered shift and b_theta
    ans_list.append([trial, theta, ans[0], B_theta])

win.close()

0 2.5987009780166286 right 4.730042335802144
1 4.867165156486368 left 180.0
2 6.921073146638825 left 53.51432017512245
3 24.03413589720658 right 4.730042335802144
4 21.56139776376556 right 15.90990257669732
5 -66.68119065411321 left 15.90990257669732
6 -7.411361730699671 left 53.51432017512245
7 57.99500976549107 left 180.0
8 21.41556722964981 right 15.90990257669732
9 25.389102956459475 right 15.90990257669732
10 -26.963456002083117 left 15.90990257669732
11 51.21949878862481 right 1.40625
12 5.852485163381676 right 15.90990257669732
13 24.446913355286267 right 53.51432017512245
14 79.75585694109606 right 4.730042335802144
15 0.3043583247208934 right 1.40625
16 34.21581175158083 left 180.0
17 36.04108263300204 left 180.0
18 -27.54324934626783 left 180.0
19 -21.407942609290057 right 180.0
20 -48.054636726709944 left 4.730042335802144
21 33.30701937199721 right 53.51432017512245
22 -33.10858807316177 right 180.0
23 -79.02986033039612 left 4.730042335802144
24 6.778780716017226 right 180

In [4]:
import pickle
pickle.dump(ans_list, open('Psychophys_discrim_%s.p' % exp_info, 'wb'))